# Compare Models

Create object to automated model comparisons

In [1]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

Using TensorFlow backend.


### ModelComparator

```
mc = ModelComparator()
mc.add_model(name, model)
mc.add_training_set(name, data)

mc.train_models(epochs, batches, **kwargs)

mc.compare_min(metric)
mc.compare_max(metric)

mc.history_for(model_name, training_set_name)
```

In [2]:
class ModelComparator:
    def __init__(self):
        self.models = {}
        self.training_sets = {}
        self.histories = {}
    
    def add_model(self, name, model):
        self.models[name] = model
    
    def add_training_set(self, name, data):
        self.training_sets[name] = data
        
    def train_models(self, **kwargs):
        for model_name in self.models:
            for tset_name in self.training_sets:
                print(f"MODEL {model_name} TRAINING ON DATASET {tset_name}")
                
                model = self.models[model_name]()
                
                history = model.fit(
                    self.training_sets[tset_name][0], self.training_sets[tset_name][1],
                    **kwargs
                )
                
                self.histories[(model_name, tset_name)] = history
    
    def compare_min(self, metric):
        result = {}
        minimum = None
        min_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amin(history[metric])
            if minimum == None or minimum >= result[hkey]:
                minimum = result[hkey]
                min_name = hkey
        return result, {"name" : min_name, "value":minimum}
            
    
    def compare_max(self, metric):
        result = {}
        maximum = None
        max_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amax(history[metric])
            if maximum == None or maximum >= result[hkey]:
                maximum = result[hkey]
                max_name = hkey
        return result, {"name" : max_name, "value": maximum}
    
    def history_for(self, model_name, training_set_name):
        return self.histories[(model_name, training_set_name)]

### Create Models

In [3]:
def nondense_model():
    # Input layer of 3 neurons 
    inp = Input(shape=(1,3))
    
    #128 layer
    d2_out = Dense(128)(inp)

    #grab first, 2nd half of the 128 layer
    d2_out_p1 = Lambda(lambda x: x[:,:,0:64])(d2_out)
    d2_out_p2 = Lambda(lambda x: x[:,:,64:128])(d2_out)

    #64 layer(s)
    d3_out = Dense(64)(d2_out_p1)
    d4_out = Dense(64)(d2_out_p2)

    #grab output nodes from both 64 layers
    d5_out = concatenate([d3_out, d4_out])
    
    o = Dense(1)(d5_out)
    
    model = Model(inp, o)

    model.compile(
        loss="MeanSquaredError",
        metrics=['accuracy']
    )
    
    return model

def dense_model_5L():
    model_5layer = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024, input_shape=(1,3)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1)
    ])
    
    model_5layer.compile(
        loss="MeanSquaredError",
        metrics=['accuracy'] #Acc not working, in testing
    )

    return model_5layer

### Datasets

In [4]:
num_examples = 20000
rstart = -50000
rend = 50000

add_setX, add_setY = ad.gen_data_add(num_examples, rstart, rend)
sub_setX, sub_setY = ad.gen_data_sub(num_examples, rstart, rend)

add_setX = add_setX.reshape(num_examples, 1, 3)
sub_setX = sub_setX.reshape(num_examples, 1, 3)

# 50-50
step = int(0.5*num_examples)

addsub_set1X = np.concatenate((add_setX[:step], sub_setX[step:]))
addsub_set1Y = np.concatenate((add_setY[:step], sub_setY[step:]))

# 80-20
step = int(0.8*num_examples)
addsub_set2X = np.concatenate((add_setX[:step], sub_setX[step:]))
addsub_set2Y = np.concatenate((add_setY[:step], sub_setY[step:]))

# MULT and DIV
mult_setX, mult_setY = ad.gen_data_mult(num_examples, rstart, rend)
div_setX, div_setY = ad.gen_data_div(num_examples, rstart, rend)

mult_setX = mult_setX.reshape(num_examples, 1, 3)
div_setX = div_setX.reshape(num_examples, 1, 3)

# MD 50-50
step = int(0.5*num_examples)
multdiv_set1X = np.concatenate((mult_setX[:step], div_setX[step:]))
multdiv_set1Y = np.concatenate((mult_setY[:step], div_setY[step:]))

# MD 80-20
step = int(0.8*num_examples)
multdiv_set2X = np.concatenate((mult_setX[:step], div_setX[step:]))
multdiv_set2Y = np.concatenate((mult_setY[:step], div_setY[step:]))

### Build ModelComparator

In [5]:
mc = ModelComparator()

mc.add_model("nondense_model", nondense_model)
mc.add_model("dense_model", dense_model_5L)

mc.add_training_set("addition", (add_setX, add_setY))
mc.add_training_set("subtraction", (sub_setX, sub_setY))
mc.add_training_set("add-sub 50/50", (addsub_set1X, addsub_set1Y))
mc.add_training_set("add-sub 80/20", (addsub_set2X, addsub_set2Y))
mc.add_training_set("multiplication", (mult_setX, mult_setY))
mc.add_training_set("division", (div_setX, div_setY))
mc.add_training_set("mult-div 50/50", (multdiv_set1X, multdiv_set1Y))
mc.add_training_set("mult-div 80/20", (multdiv_set2X, multdiv_set2Y))



### Train Models

In [6]:
num_epochs = 100
batch_size = num_examples

mc.train_models(
    batch_size=batch_size,
    epochs=num_epochs
)

MODEL nondense_model TRAINING ON DATASET addition
Epoch 1/100
1/1 [==============================] - 0s 31ms/step - loss: 1691492608.0000 - accuracy: 5.0000e-05
Epoch 2/100
1/1 [==============================] - 0s 5ms/step - loss: 977951488.0000 - accuracy: 5.0000e-05
Epoch 3/100
1/1 [==============================] - 0s 7ms/step - loss: 621272896.0000 - accuracy: 5.0000e-05
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 397352096.0000 - accuracy: 5.0000e-05
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 249457328.0000 - accuracy: 5.0000e-05
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: 151641760.0000 - accuracy: 5.0000e-05
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 88397224.0000 - accuracy: 5.0000e-05
Epoch 8/100
1/1 [==============================] - 0s 7ms/step - loss: 49032808.0000 - accuracy: 5.0000e-05
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 2571372

1/1 [==============================] - 0s 13ms/step - loss: 5419871.5000 - accuracy: 5.0000e-05
Epoch 78/100
1/1 [==============================] - 0s 3ms/step - loss: 4490329.5000 - accuracy: 0.0000e+00
Epoch 79/100
1/1 [==============================] - 0s 4ms/step - loss: 3480834.7500 - accuracy: 5.0000e-05
Epoch 80/100
1/1 [==============================] - 0s 3ms/step - loss: 3205521.5000 - accuracy: 0.0000e+00
Epoch 81/100
1/1 [==============================] - 0s 2ms/step - loss: 3286456.0000 - accuracy: 5.0000e-05
Epoch 82/100
1/1 [==============================] - 0s 16ms/step - loss: 4126704.7500 - accuracy: 5.0000e-05
Epoch 83/100
1/1 [==============================] - 0s 12ms/step - loss: 4898742.5000 - accuracy: 0.0000e+00
Epoch 84/100
1/1 [==============================] - 0s 3ms/step - loss: 5537503.5000 - accuracy: 5.0000e-05
Epoch 85/100
1/1 [==============================] - 0s 15ms/step - loss: 4717492.0000 - accuracy: 0.0000e+00
Epoch 86/100
1/1 [===================

Epoch 53/100
1/1 [==============================] - 0s 7ms/step - loss: 4026811.2500 - accuracy: 0.0000e+00
Epoch 54/100
1/1 [==============================] - 0s 3ms/step - loss: 3252648.2500 - accuracy: 0.0000e+00
Epoch 55/100
1/1 [==============================] - 0s 2ms/step - loss: 2615104.2500 - accuracy: 0.0000e+00
Epoch 56/100
1/1 [==============================] - 0s 4ms/step - loss: 2209785.7500 - accuracy: 0.0000e+00
Epoch 57/100
1/1 [==============================] - 0s 5ms/step - loss: 1989252.8750 - accuracy: 0.0000e+00
Epoch 58/100
1/1 [==============================] - 0s 5ms/step - loss: 1954600.1250 - accuracy: 0.0000e+00
Epoch 59/100
1/1 [==============================] - 0s 3ms/step - loss: 2067131.1250 - accuracy: 0.0000e+00
Epoch 60/100
1/1 [==============================] - 0s 3ms/step - loss: 2385029.5000 - accuracy: 0.0000e+00
Epoch 61/100
1/1 [==============================] - 0s 4ms/step - loss: 2911751.7500 - accuracy: 0.0000e+00
Epoch 62/100
1/1 [==========

1/1 [==============================] - 0s 5ms/step - loss: 836744640.0000 - accuracy: 0.0000e+00
Epoch 28/100
1/1 [==============================] - 0s 0s/step - loss: 836810688.0000 - accuracy: 0.0000e+00
Epoch 29/100
1/1 [==============================] - 0s 3ms/step - loss: 836852672.0000 - accuracy: 0.0000e+00
Epoch 30/100
1/1 [==============================] - 0s 0s/step - loss: 836929536.0000 - accuracy: 0.0000e+00
Epoch 31/100
1/1 [==============================] - 0s 0s/step - loss: 836982272.0000 - accuracy: 0.0000e+00
Epoch 32/100
1/1 [==============================] - 0s 7ms/step - loss: 837070336.0000 - accuracy: 0.0000e+00
Epoch 33/100
1/1 [==============================] - 0s 3ms/step - loss: 837126656.0000 - accuracy: 0.0000e+00
Epoch 34/100
1/1 [==============================] - 0s 9ms/step - loss: 837213376.0000 - accuracy: 0.0000e+00
Epoch 35/100
1/1 [==============================] - 0s 3ms/step - loss: 837258816.0000 - accuracy: 0.0000e+00
Epoch 36/100
1/1 [========

1/1 [==============================] - 0s 0s/step - loss: 1491997184.0000 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 0s/step - loss: 948893888.0000 - accuracy: 5.0000e-05
Epoch 3/100
1/1 [==============================] - 0s 0s/step - loss: 735358272.0000 - accuracy: 5.0000e-05
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 630870848.0000 - accuracy: 5.0000e-05
Epoch 5/100
1/1 [==============================] - 0s 4ms/step - loss: 579141184.0000 - accuracy: 5.0000e-05
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 554761792.0000 - accuracy: 5.0000e-05
Epoch 7/100
1/1 [==============================] - 0s 7ms/step - loss: 544144576.0000 - accuracy: 5.0000e-05
Epoch 8/100
1/1 [==============================] - 0s 3ms/step - loss: 539938944.0000 - accuracy: 5.0000e-05
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 538435776.0000 - accuracy: 5.0000e-05
Epoch 10/100
1/1 [===============

...

1/1 [==============================] - 0s 0s/step - loss: 168933.3750 - accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 0s/step - loss: 86340.1328 - accuracy: 0.0000e+00
Epoch 8/100
1/1 [==============================] - 0s 16ms/step - loss: 79169.0781 - accuracy: 0.0000e+00
Epoch 9/100
1/1 [==============================] - 0s 0s/step - loss: 78691.0469 - accuracy: 0.0000e+00
Epoch 10/100
1/1 [==============================] - 0s 16ms/step - loss: 78665.6094 - accuracy: 0.0000e+00
Epoch 11/100
1/1 [==============================] - 0s 16ms/step - loss: 78661.8828 - accuracy: 0.0000e+00
Epoch 12/100
1/1 [==============================] - 0s 0s/step - loss: 78658.9453 - accuracy: 0.0000e+00
Epoch 13/100
1/1 [==============================] - 0s 0s/step - loss: 78656.0156 - accuracy: 0.0000e+00
Epoch 14/100
1/1 [==============================] - 0s 0s/step - loss: 78653.0859 - accuracy: 0.0000e+00
Epoch 15/100
1/1 [==============================] - 0s 3ms/step 

1/1 [==============================] - 0s 0s/step - loss: 969903.9375 - accuracy: 0.0000e+00
Epoch 84/100
1/1 [==============================] - 0s 0s/step - loss: 316267.6250 - accuracy: 0.0000e+00
Epoch 85/100
1/1 [==============================] - 0s 0s/step - loss: 140338.1875 - accuracy: 0.0000e+00
Epoch 86/100
1/1 [==============================] - 0s 16ms/step - loss: 105581.9375 - accuracy: 0.0000e+00
Epoch 87/100
1/1 [==============================] - 0s 0s/step - loss: 92612.0703 - accuracy: 0.0000e+00
Epoch 88/100
1/1 [==============================] - 0s 0s/step - loss: 88550.1016 - accuracy: 0.0000e+00
Epoch 89/100
1/1 [==============================] - 0s 0s/step - loss: 87005.3359 - accuracy: 0.0000e+00
Epoch 90/100
1/1 [==============================] - 0s 0s/step - loss: 87708.7734 - accuracy: 0.0000e+00
Epoch 91/100
1/1 [==============================] - 0s 0s/step - loss: 90251.5547 - accuracy: 0.0000e+00
Epoch 92/100
1/1 [==============================] - 0s 0s/step

1/1 [==============================] - 0s 0s/step - loss: 348568310108913664.0000 - accuracy: 0.0000e+00
Epoch 54/100
1/1 [==============================] - 0s 0s/step - loss: 348568413188128768.0000 - accuracy: 0.0000e+00
Epoch 55/100
1/1 [==============================] - 0s 4ms/step - loss: 348568310108913664.0000 - accuracy: 0.0000e+00
Epoch 56/100
1/1 [==============================] - 0s 16ms/step - loss: 348568413188128768.0000 - accuracy: 0.0000e+00
Epoch 57/100
1/1 [==============================] - 0s 16ms/step - loss: 348568344468652032.0000 - accuracy: 0.0000e+00
Epoch 58/100
1/1 [==============================] - 0s 0s/step - loss: 348568378828390400.0000 - accuracy: 0.0000e+00
Epoch 59/100
1/1 [==============================] - 0s 0s/step - loss: 348568275749175296.0000 - accuracy: 0.0000e+00
Epoch 60/100
1/1 [==============================] - 0s 16ms/step - loss: 348568378828390400.0000 - accuracy: 0.0000e+00
Epoch 61/100
1/1 [==============================] - 0s 0s/step

1/1 [==============================] - 0s 0s/step - loss: 562151364382162944.0000 - accuracy: 5.0000e-05
Epoch 22/100
1/1 [==============================] - 0s 0s/step - loss: 562151673619808256.0000 - accuracy: 5.0000e-05
Epoch 23/100
1/1 [==============================] - 0s 0s/step - loss: 562151364382162944.0000 - accuracy: 5.0000e-05
Epoch 24/100
1/1 [==============================] - 0s 0s/step - loss: 562151811058761728.0000 - accuracy: 5.0000e-05
Epoch 25/100
1/1 [==============================] - 0s 0s/step - loss: 562151364382162944.0000 - accuracy: 5.0000e-05
Epoch 26/100
1/1 [==============================] - 0s 0s/step - loss: 562151811058761728.0000 - accuracy: 5.0000e-05
Epoch 27/100
1/1 [==============================] - 0s 0s/step - loss: 562151364382162944.0000 - accuracy: 5.0000e-05
Epoch 28/100
1/1 [==============================] - 0s 0s/step - loss: 562151811058761728.0000 - accuracy: 5.0000e-05
Epoch 29/100
1/1 [==============================] - 0s 0s/step - loss

1/1 [==============================] - 0s 0s/step - loss: 562151467461378048.0000 - accuracy: 5.0000e-05
Epoch 91/100
1/1 [==============================] - 0s 1ms/step - loss: 562151123863994368.0000 - accuracy: 5.0000e-05
Epoch 92/100
1/1 [==============================] - 0s 2ms/step - loss: 562151467461378048.0000 - accuracy: 5.0000e-05
Epoch 93/100
1/1 [==============================] - 0s 16ms/step - loss: 562151226943209472.0000 - accuracy: 5.0000e-05
Epoch 94/100
1/1 [==============================] - 0s 8ms/step - loss: 562151536180854784.0000 - accuracy: 5.0000e-05
Epoch 95/100
1/1 [==============================] - 0s 0s/step - loss: 562151192583471104.0000 - accuracy: 5.0000e-05
Epoch 96/100
1/1 [==============================] - 0s 0s/step - loss: 562151467461378048.0000 - accuracy: 5.0000e-05
Epoch 97/100
1/1 [==============================] - 0s 0s/step - loss: 562151192583471104.0000 - accuracy: 5.0000e-05
Epoch 98/100
1/1 [==============================] - 0s 0s/step -

### compare min loss

In [7]:
models_min_loss, min_loss = mc.compare_min("loss")

In [8]:
models_min_loss

{('nondense_model', 'addition'): 19.66743278503418,
 ('nondense_model', 'subtraction'): 5.0636444091796875,
 ('nondense_model', 'add-sub 50/50'): 834761088.0,
 ('nondense_model', 'add-sub 80/20'): 537757568.0,
 ('nondense_model', 'multiplication'): 7.045452670047355e+17,
 ('nondense_model', 'division'): 645031.5,
 ('nondense_model', 'mult-div 50/50'): 3.485711619671982e+17,
 ('nondense_model', 'mult-div 80/20'): 5.6217335461471846e+17,
 ('dense_model', 'addition'): 1020197.8125,
 ('dense_model', 'subtraction'): 147496.8125,
 ('dense_model', 'add-sub 50/50'): 834507712.0,
 ('dense_model', 'add-sub 80/20'): 537748032.0,
 ('dense_model', 'multiplication'): 7.045364709117133e+17,
 ('dense_model', 'division'): 78616.9375,
 ('dense_model', 'mult-div 50/50'): 3.485680352310067e+17,
 ('dense_model', 'mult-div 80/20'): 5.6215091770556416e+17}